# Variant segmentation and leakage review

In [1]:
import sys  # no installation needed for stdlib/project-local
from pathlib import Path  # no installation needed for stdlib/project-local

ROOT = Path(r"C:\\Users\\quantbase\\Desktop\\ecom_forecast")
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))


In [2]:
import pandas as pd  # already in env ? no new install for third-party libs

from src.config import ProjectPaths  # no installation needed for stdlib/project-local
from src.segmentation import (  # no installation needed for stdlib/project-local
    bucket_variants,
    category_summary,
    prepare_variant_metrics,
    top_leaks,
)


In [3]:
paths = ProjectPaths.from_root()
paths.ensure_directories()
seg_dir = paths.outputs_dir / 'segmentation'
seg_dir.mkdir(parents=True, exist_ok=True)
variant = pd.read_pickle(paths.clean_dir / 'variant.pkl')


In [4]:
variant_metrics = prepare_variant_metrics(variant)
variant_bucketed = bucket_variants(variant_metrics)
category_stats = category_summary(variant_bucketed)
leaks = top_leaks(variant_bucketed, n=20)
if variant_bucketed['category'].nunique() == 1 and variant_bucketed['category'].iloc[0] == 'Mid':
    raise ValueError('Segmentation produced only Mid category; check input distributions.')
category_stats


,category,variants,net_sales,returns_abs,gross_margin_d,gm_leak,share_net_sales,share_returns,share_gm_leakage
0,High-Margin Staple,1159,392278.58,0.00,259129.59,0.00,0.144632,0.000000,0.000000
1,High-Return Item,1830,607923.21,455923.34,353375.76,4621.41,0.224140,0.835944,0.254090
2,Low-Margin Promo,1434,518993.54,11326.62,263555.96,1365.00,0.191352,0.020768,0.075049
3,Mid,2796,1193052.24,78149.72,709943.32,12201.70,0.439876,0.143289,0.670861


In [5]:
variant_bucketed.to_csv(seg_dir / 'variant_metrics.csv', index=False)
category_stats.to_csv(seg_dir / 'category_summary.csv', index=False)
leaks['top_returns_abs'].to_csv(seg_dir / 'top_returns_abs.csv', index=False)
leaks['top_return_rate'].to_csv(seg_dir / 'top_return_rate.csv', index=False)
leaks['worst_gross_margin'].to_csv(seg_dir / 'worst_gross_margin.csv', index=False)
['variant_metrics.csv', 'category_summary.csv', 'top_returns_abs.csv', 'top_return_rate.csv', 'worst_gross_margin.csv']


['variant_metrics.csv',
 'category_summary.csv',
 'top_returns_abs.csv',
 'top_return_rate.csv',
 'worst_gross_margin.csv']